Firstly, I have stated all of the imports necessary for this project at the very beginning to keep them together.

Included in this is the comet_ml package that I am using to track the performance of the models as I develop them further, helping to improve their performance. 

In [ ]:
import torch
import os
import getpass

from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt

from torchmetrics import ConfusionMatrix

import torch.nn as nn
import torch.optim as optim
import comet_ml

import numpy as np

comet_api_key=os.environ.get("COMET_API_KEY")
if comet_api_key is None:
  comet_api_key=getpass.getpass("Enter key")


To begin with, the images need to be resized to 180x180. The compose function enables us to carry out two transforms at the same time, resizing the images and converting them into tensors that PyTorch can work with. These transforms are first defined, before the reading of the dataset. 

The location of this dataset is within the same directory as the jupyter notebook file, making it easier to access. When reading the image dataset, the defined transform is applied to all of the images immediately. 

In [ ]:
transform = transforms.Compose([transforms.Resize((180, 180)),
                                transforms.ToTensor()])

dataset = datasets.ImageFolder(root='data/images_original/', transform=transform)

The following code carries out the splitting of the dataset into the respecitve training, validation and testing sets. For this project, 70% of the dataset is used for training, 20% is used for validation and 10% is used for testing.

The function 'random_split' essentially shuffles the dataset while it is being split to allow all three sets to have a balanced representation of each class for most effective training. The manual seed is also set to ensure the shuffling of the data is consistent so that training sessions can be compared more fairly, reducing the variance in the data being trained.

In [ ]:
total_size = len(dataset)
print(f'Dataset size: {total_size}')

train_size = int(0.7 * total_size)
val_size = int(0.2 * total_size)
test_size = total_size - train_size - val_size

print(f'Training set size: {train_size}')
print(f'Validation set size: {val_size}')
print(f'Testing set size: {test_size}')

torch.manual_seed(42)

train_set, val_set, test_set = random_split(dataset, [train_size, val_size, test_size])

To visualise examples of the dataset to get a better understanding of the data, this code plots 10 examples within the training set with their respective labels. As the training set is already shuffled, this visualisation should display images from a variety of the genre classes. 

The 10 tensors to be visualised are converted back into an image format by first converting to a numpy array and then transposing the dimensions for compaitibility with matplotlib. 

In [ ]:
itr = iter(train_set)
fig = plt.figure()
fig.tight_layout()
plt.subplots_adjust(wspace=0.7, hspace=0.7)

for i in range(10):
    img, label = next(itr)
    img_np = np.transpose(img.numpy(), (1, 2, 0))
    sub = fig.add_subplot(4, 5, i + 1)
    sub.set_title(str(label))
    sub.axes.get_xaxis().set_visible(False)
    sub.axes.get_yaxis().set_visible(False)
    plt.imshow(img_np)

plt.show()

Batches are required to be able to train the model, as it is not suitable to load the entire dataset into memory for training all at once. The batch size of 32 has been chosen to balance the effective use of GPU acceleration with the memeory constraints of having a high batch size. 

Each of the datasets are then loaded into a DataLoader. This helps to iterate through the data in batches of the size specified for training, validation and testing. 

In [ ]:
batch_size = 32

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

This is the definition of the first model, of which is a fully connected network with two hidden layers (fc1 and fc2) and an output layer (fc3). Both of these hidden layers use ReLU as the activation function as it is widely used and computationally efficient.

The output layer specifies 10 output features, of which represent the 10 classes of music genres. This therefore makes use of the cross entropy loss function that implicitly uses softmax. 

The flatten function is used to flatten the image for use in the input layer. 

In [ ]:
class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(3 *180 * 180, 512)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(512, 256)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.flatten(x)
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.fc3(x)
        return x


The second model is a convolutional network that follows the structure from figure 1. This starts with two convolutional layers, both using ReLU as the activation function, followed by a max pooling layer. This follows into another two convolutional layers, again using ReLU activation functions before using two fully connected layers. The first is a hidden layer that also uses ReLU, with an output of 256 features and the second one being the output layer. 

In [ ]:
class Net2(nn.Module):
    def __init__(self):
        super(Net2, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=(3,3), stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(32, 32, kernel_size=(2, 2), stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=(3, 3), stride=1, padding=1)
        self.relu3 = nn.ReLU()
        self.conv4 = nn.Conv2d(64, 128, kernel_size=(3, 3), stride=1, padding=1)
        self.relu4 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * 45 * 45, 256)
        self.relu5 = nn.ReLU()
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.relu1(self.conv1(x))
        x = self.relu2(self.conv2(x))
        x = self.pool1(x)
        x = self.relu3(self.conv3(x))
        x = self.relu4(self.conv4(x))
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.relu5(self.fc1(x))
        x = self.fc2(x)
        return x

The third model is the same as the previous convolutional network with the addition of a single batch normalisation layer after the input layer. This is to help prevent the model from overfitting to the training set in order to provide a good predictive performance on unseen data. 

In [ ]:
class Net3(nn.Module):
    def __init__(self):
        super(Net3, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=(3,3), stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(32, 32, kernel_size=(2, 2), stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=(3, 3), stride=1, padding=1)
        self.bn = nn.BatchNorm2d(64)
        self.relu3 = nn.ReLU()
        self.conv4 = nn.Conv2d(64, 128, kernel_size=(3, 3), stride=1, padding=1)
        self.relu4 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * 45 * 45, 256)
        self.relu5 = nn.ReLU()
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.relu1(self.conv1(x))
        x = self.relu2(self.conv2(x))
        x = self.pool1(x)
        x = self.relu3(self.bn(self.conv3(x)))
        x = self.relu4(self.conv4(x))
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.relu5(self.fc1(x))
        x = self.fc2(x)
        return x

To record the performance of each model, I am using comet_ml to track each training experiment and its outcome. This helps especially when tweaking the hyperparameters in order to provide improved model performance. This includes the logging of some hyperparameters, namely learning rate and batch size, along with performance metrics such as the loss.

The below code creates a new experiment for me to record this session within.

In [ ]:
experiment = comet_ml.Experiment(api_key=comet_api_key, workspace="benhipwell", project_name="cw1", auto_metric_logging=True, auto_output_logging=True, experiment_key="benhipwellcomp6252genreclassification2")
experiment.set_name('benhipwellcomp6252genreclassification2')

# conf_matr=ConfusionMatrix(task='multiclass', num_classes=10)
# conf_matr=conf_matr.to('cuda')

Instead of always creating a new experiment, this code can be used to carry on an experiment that has already been started. This has been very useful to keep training sessions on jupyter notebook separate whilst still being able to combine them within the same experiment for comparison.

In [ ]:
experiment = comet_ml.ExistingExperiment(api_key=comet_api_key, experiment_key="benhipwellcomp6252genreclassification2")

This is the train function, containing the main training loop. This takes the model, from one of those defined earlier, the training loader, the optimizer, the loss function and the number of epochs to run. After each epoch, it displays the current loss to give an estimation on how well the model is training. 

In [ ]:
import torch.autograd.profiler as profiler

def train(model, train_loader, optimizer, loss_func, epochs, name):
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.cuda(), labels.cuda()
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_func(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()


        print(f"epoch {epoch + 1}/{epochs}, loss: {running_loss / len(train_loader)}")
        experiment.log_metrics({f'loss_{name}': running_loss}, epoch=epoch)       


    experiment.log_metrics({f'final_loss_{name}': (running_loss / len(train_loader))})     

The evaluation function takes the trained model, the validation or test set and the loss function. This function measures how well the trained model performs when predicting new, unseen data. It returns the average loss and accuracy of the predictions as well as recording the results to the confusion matrix. 

In [ ]:
def evaluate(model, data_loader, loss_func, name):
    
    print(f"evaluating: {name}")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    model.eval()

    total_correct = 0
    total_samples = 0
    total_loss = 0.0

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = loss_func(outputs, labels)

            _, predicted = torch.max(outputs, 1)
            total_correct += (predicted == labels).sum().item()
            total_samples += labels.size(0)
            total_loss += loss.item()
            # conf_matr.update(predicted,labels)

    accuracy = total_correct / total_samples
    avg_loss = total_loss / len(data_loader)

    print(f"accuracy: {accuracy}, average loss: {avg_loss}")
    experiment.log_metrics({f"test_accuracy_{name}": accuracy})


This instantiates the models based on the previously defined networks, Net1, Net2 and Net3. The fourth network only uses a different optimizer and therefore does not require its own definition. 

Here the optimizer and loss function is decided upon, using Adam for the majority of the models, other then Net4 which requires RMSprop. The learning rate here is also set accordingly, as well as the loss function. I have used Cross Entropy Loss as it is a widely used loss function for multi-class classification and has performed well. 

In [ ]:
lr = 0.00003
loss_func = nn.CrossEntropyLoss()

Each of the models are then trained and evaluated using the defined train and evaluate functions. This is completed for each model, each for 50 and 100 epochs and then evaluated on both the validation and test sets. Each is given a name so that they can be identified and compared on the comet_ml graphs. 

I have also decided to run them individually after discovering that attempting to run multiple training sessions consecutively effects training speed. 

To perform the training and evaluation of Net1 with 50 epochs

In [ ]:
net1_50 = Net1().cuda()
optimizer1_50 = optim.Adam(net1_50.parameters(), lr=lr)
train(net1_50, train_loader, optimizer1_50, loss_func, epochs=50, name='net1_50')
evaluate(net1_50, test_loader, loss_func, name='net1_50_test')

To perform the training and evaluation of Net2 with 50 epochs

In [ ]:
net2_50 = Net2().cuda()
optimizer2_50 = optim.Adam(net2_50.parameters(), lr=lr)
train(net2_50, train_loader, optimizer2_50, loss_func, epochs=50, name='net2_50')
evaluate(net2_50, test_loader, loss_func, name='net2_50_test')

To perform the training and evaluation of Net3 with 50 epochs

In [ ]:
net3_50 = Net3().cuda()
optimizer3_50 = optim.Adam(net3_50.parameters(), lr=lr)
train(net3_50, train_loader, optimizer3_50, loss_func, epochs=50, name='net3_50')
evaluate(net3_50, test_loader, loss_func, name='net3_50_test')

To perform the training and evaluation of Net4 with 50 epochs

In [ ]:
net4_50 = Net3().cuda()
optimizer4_50 = optim.Adam(net4_50.parameters(), lr=lr)
train(net4_50, train_loader, optimizer4_50, loss_func, epochs=50, name='net4_50')
evaluate(net4_50, test_loader, loss_func, name='net4_50_test')

To perform the training and evaluation of Net1 with 100 epochs

In [ ]:
net1_100 = Net1().cuda()
optimizer1_100 = optim.Adam(net1_100.parameters(), lr=lr)
train(net1_100, train_loader, optimizer1_100, loss_func, epochs=100, name='net1_100')
evaluate(net1_100, test_loader, loss_func, name='net1_100_test')

To perform the training and evaluation of Net2 with 100 epochs

In [ ]:
net2_100 = Net2().cuda()
optimizer2_100 = optim.Adam(net2_100.parameters(), lr=lr)
train(net2_100, train_loader, optimizer2_100, loss_func, epochs=100, name='net2_100')
evaluate(net2_100, test_loader, loss_func, name='net2_100_test')

To perform the training and evaluation of Net3 with 100 epochs

In [ ]:
net3_100 = Net3().cuda()
optimizer3_100 = optim.Adam(net3_100.parameters(), lr=lr)
train(net3_100, train_loader, optimizer3_100, loss_func, epochs=100, name='net3_100')
evaluate(net3_100, test_loader, loss_func, name='net3_100_test')

To perform the training and evaluation of Net4 with 100 epochs

In [ ]:
net4_100 = Net3().cuda()
optimizer4_100 = optim.RMSprop(net4_100.parameters(), lr=lr)
train(net4_100, train_loader, optimizer4_100, loss_func, epochs=100, name='net4_100')
evaluate(net4_100, test_loader, loss_func, name='net4_100_test')

This finalises the comet_ml experiment logging

In [ ]:
# x=conf_matr.compute().cpu().numpy()
# experiment.log_confusion_matrix(matrix=x)

params = {
    "batch_size":batch_size,
    "learning_rate":lr,
}

experiment.log_parameters(params)


experiment.end()